In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [4]:
prototxtPath=os.path.sep.join([r'C:\Users\Harsh\Desktop\Major\face detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\Harsh\Desktop\Major\face detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [5]:
prototxtPath

'C:\\Users\\Harsh\\Desktop\\Major\\face detector\\deploy.prototxt'

In [6]:

weightsPath

'C:\\Users\\Harsh\\Desktop\\Major\\face detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [7]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [8]:
model=load_model(r'C:\Users\Harsh\Desktop\Major\mobilenet_v2.model')

In [9]:
#image=cv2.imread(r'C:\Users\asus\Desktop\Realworl mask detection\train\with_mask\0230.jpg')
#image = cv2.imread("maskman.jpg", 0)

In [10]:
image = cv2.imread(r'C:\Users\Harsh\Desktop\Major\harsh3.jpeg')
resized = cv2.resize(image, (224, 224))
blob = cv2.dnn.blobFromImage(resized, 1, (224, 224), (104, 117, 123))
print("First Blob: {}".format(blob.shape))

First Blob: (1, 3, 224, 224)


In [11]:
image

array([[[221, 218, 220],
        [221, 218, 220],
        [221, 218, 220],
        ...,
        [210, 209, 213],
        [210, 209, 213],
        [210, 209, 213]],

       [[221, 218, 220],
        [221, 218, 220],
        [221, 218, 220],
        ...,
        [210, 209, 213],
        [210, 209, 213],
        [210, 209, 213]],

       [[221, 218, 220],
        [221, 218, 220],
        [221, 218, 220],
        ...,
        [210, 209, 213],
        [210, 209, 213],
        [210, 209, 213]],

       ...,

       [[ 63,  63,  49],
        [ 50,  51,  35],
        [ 51,  48,  33],
        ...,
        [ 43,  50,  45],
        [ 64,  75,  72],
        [138, 151, 149]],

       [[ 63,  63,  49],
        [ 50,  51,  35],
        [ 51,  48,  33],
        ...,
        [ 41,  48,  43],
        [ 52,  61,  58],
        [136, 147, 145]],

       [[ 64,  64,  50],
        [ 51,  52,  36],
        [ 51,  48,  33],
        ...,
        [ 40,  47,  42],
        [ 45,  54,  51],
        [133, 144, 142]]

In [12]:
(h,w)=image.shape[:2]


In [13]:
(h,w)


(1152, 528)

In [14]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))


In [15]:
blob


array([[[[ 117.,  117.,  117., ...,  106.,  106.,  106.],
         [ 117.,  117.,  117., ...,  106.,  106.,  106.],
         [ 118.,  118.,  118., ...,  107.,  107.,  107.],
         ...,
         [ -41.,  -62.,  -56., ...,  -65.,  -20.,   28.],
         [ -39.,  -53.,  -53., ...,  -54.,  -55.,   25.],
         [ -46.,  -53.,  -55., ...,  -59.,  -62.,    2.]],

        [[  41.,   41.,   41., ...,   32.,   32.,   32.],
         [  41.,   41.,   41., ...,   32.,   32.,   32.],
         [  42.,   42.,   42., ...,   33.,   33.,   33.],
         ...,
         [-113., -136., -129., ..., -134.,  -84.,  -33.],
         [-112., -127., -127., ..., -124., -119.,  -36.],
         [-119., -129., -130., ..., -130., -128.,  -59.]],

        [[  97.,   97.,   97., ...,   90.,   90.,   90.],
         [  97.,   97.,   97., ...,   90.,   90.,   90.],
         [  98.,   98.,   98., ...,   91.,   91.,   91.],
         ...,
         [ -68.,  -92.,  -81., ...,  -85.,  -30.,   25.],
         [ -70.,  -87.,  -

In [16]:
blob.shape


(1, 3, 300, 300)

In [17]:
net.setInput(blob)
detections=net.forward()

In [18]:
detections


array([[[[0.        , 1.        , 0.99950206, ..., 0.32041913,
          0.6286717 , 0.5826885 ],
         [0.        , 1.        , 0.12095279, ..., 4.008711  ,
          4.842184  , 4.9842367 ],
         [0.        , 1.        , 0.11460672, ..., 0.12265658,
          0.98336685, 0.8733761 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [19]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()